In [2]:
%pip install gdown

train = '1x_D1Feh88EFHQ63SEe55sAymUnztqzeK'
val = '1CcvUQxEuzPbwLOYjA1rxmyBxMF_au6wZ'
test = '1Kqf7-0Jkp1doGJWyhgxW1ku738IO4xji'

train_embeddings_per_word = '1kc018iaKNu8Y1DJd4WfyxWOxTQNNYt-h'
val_embeddings_per_word = '1--fg57qeqXEFrLqYvoVcN_9-o-Pzc26I'
test_embeddings_per_word = '1--K6JKVyn8UE9oo-4wzf5B6IzJ0JrI7c'

train_embeddings_per_sentence = '1-5dkTRFtcYbJJf5AWufBxgMba8FRSN7i'
val_embeddings_per_sentence = '1-FwXE-za1j3cBk9Cvw5T3DnzkJAGiY-M'
test_embeddings_per_sentence = '1-8icZrYP0oNUTnh6mwSGwK9fLAS1gcQU'

uncleaned_train_embedding = '13UONDopwe65QnuMK6jJWQUIdyx3OI-FA'
uncleaned_val_embedding = '1-0eCfZ8xSvLAI4RuC2e1WLf_Z09GmAaN'
uncleaned_test_embedding = '1-0CfuTbiTOIfDr4R76MO4Hd7GVeFlSrn'

model_path = '1V9PmKPW7kLwpHGNk6xhEKlB2QWQ3BTNG'

!gdown {train}
!gdown {val}
!gdown {test}

!gdown {uncleaned_train_embedding}
!gdown {uncleaned_val_embedding}
!gdown {uncleaned_test_embedding}

Note: you may need to restart the kernel to use updated packages.
Downloading...
From: https://drive.google.com/uc?id=1x_D1Feh88EFHQ63SEe55sAymUnztqzeK
To: /kaggle/working/train.pkl
100%|███████████████████████████████████████| 3.59M/3.59M [00:00<00:00, 149MB/s]
Downloading...
From: https://drive.google.com/uc?id=1CcvUQxEuzPbwLOYjA1rxmyBxMF_au6wZ
To: /kaggle/working/val.pkl
100%|█████████████████████████████████████████| 769k/769k [00:00<00:00, 114MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Kqf7-0Jkp1doGJWyhgxW1ku738IO4xji
To: /kaggle/working/test.pkl
100%|████████████████████████████████████████| 762k/762k [00:00<00:00, 94.8MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=13UONDopwe65QnuMK6jJWQUIdyx3OI-FA
From (redirected): https://drive.google.com/uc?id=13UONDopwe65QnuMK6jJWQUIdyx3OI-FA&confirm=t&uuid=ca791446-209f-4664-a318-23c22268e34f
To: /kaggle/working/train_embeddings_per_word.pkl
100%|███████████████████████████████████████| 2.16G/2.16G [00:0

In [3]:
import pandas as pd
import numpy as np
import pickle
import torch
import shap
import seaborn as sns
import tensorflow as tf
import keras
import keras_tuner
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GRU
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from lime.lime_text import LimeTextExplainer
from transformers import BertModel, AutoTokenizer, TFAutoModel
from time import time

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/o

In [4]:
classes = ['none', 'anger', 'joy', 'sadness', 'love', 'sympathy', 'surprise', 'fear']

with open('/kaggle/working/train.pkl', 'rb') as f:
    train = pickle.load(f)
    
with open('/kaggle/working/val.pkl', 'rb') as f:
    val = pickle.load(f)

with open('/kaggle/working/test.pkl', 'rb') as f:
    test = pickle.load(f)

with open('/kaggle/working/train_embeddings_per_word.pkl', 'rb') as f:
    train_embeddings = pickle.load(f)
    
with open('/kaggle/working/val_embeddings_per_word.pkl', 'rb') as f:
    val_embeddings = pickle.load(f)
    
with open('/kaggle/working/test_embeddings_per_word.pkl', 'rb') as f:
    test_embeddings = pickle.load(f)

In [5]:
encoder = OneHotEncoder()

y_train = encoder.fit_transform(train['label'].values.reshape(-1,1)).toarray()
y_val = encoder.transform(val['label'].values.reshape(-1,1)).toarray()
y_test = encoder.transform(test['label'].values.reshape(-1,1)).toarray()

In [6]:
max_length = 50

In [7]:
X_train = train_embeddings
X_val = val_embeddings
X_test = test_embeddings

In [8]:
BATCH_SIZE = 128
EPOCHS = 50
EMBED_SIZE = 300
LEARNING_RATE =  0.001

early_stopping_monitor = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    min_delta=0,
    patience=10,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True)

In [9]:
def build_gru(hp):
    model = Sequential()
    gru_units = hp.Int('gru_base_units', min_value=32, max_value=256, step=2, sampling='log')
    gru_layers = hp.Int('num_gru_layers', 0, 5)
    gru_dropout = hp.Float('gru_dropout', min_value=0, max_value=0.5)
    model.add(tf.keras.layers.GRU(gru_units*(2**gru_layers), return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(gru_dropout))
    for i in range(gru_layers): 
        model.add(tf.keras.layers.GRU(gru_units*(2**(gru_layers-i)), return_sequences=(i < gru_layers - 1)))
        model.add(Dropout(gru_dropout))

    for i in range(hp.Int('num_dense_layers', 0, 3)):
        model.add(Dense(units=gru_units/(2**i)))
        model.add(Dropout(hp.Float('dense_dropout', min_value=0, max_value=0.5)))   

    model.add(Dense(8, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2)),
                loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [10]:
def build_gru_2(hp):
    model = Sequential()
    model.add(tf.keras.layers.GRU(hp.Int('gru_units', min_value=32, max_value=1024, step=2, sampling='log'), return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(hp.Float('dense_dropout', min_value=0, max_value=0.5)))
    model.add(Dense(8, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2)),
                loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [11]:
def build_gru_3(hp):
    model = Sequential()
    gru_units = hp.Int('gru_base_units', min_value=32, max_value=256, step=2, sampling='log')
    gru_dropout = hp.Float('dropout', min_value=0, max_value=0.5)
    model.add(GRU(4*gru_units, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(gru_dropout))
    
    model.add(GRU(2*gru_units, return_sequences=True))
    model.add(Dropout(gru_dropout))
    
    model.add(GRU(gru_units, return_sequences=False))
    model.add(Dropout(gru_dropout))
    
    model.add(Dense(8, activation='softmax'))

    model.compile(optimizer=Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2)),
                loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [12]:
class Custom_Bayesian(keras_tuner.tuners.BayesianOptimization):
  def run_trial(self, trial, *args, **kwargs):
    kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 16, 512, step=2, sampling='log')
    return super(Custom_Bayesian, self).run_trial(trial, *args, **kwargs)


In [13]:
tuner = Custom_Bayesian(
    build_gru,
    objective='val_accuracy',
    max_trials=20,
    overwrite=True,
    directory='my_dir2',
    project_name='gru')

In [ ]:
tuner.search(X_train, y_train, validation_data=(X_test, y_test), epochs=50, callbacks=[tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10)])

Trial 2 Complete [00h 12m 44s]

Best val_accuracy So Far: 0.7596026659011841
Total elapsed time: 00h 15m 29s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
128               |128               |gru_base_units
0                 |2                 |num_gru_layers
0.29498           |0.36979           |gru_dropout
1                 |3                 |num_dense_layers
0.0028939         |0.0021535         |learning_rate
512               |16                |batch_size
0.30236           |0                 |dense_dropout



In [ ]:
tuner.get_best_hyperparameters()[0].values

In [15]:
tuner2 = Custom_Bayesian(
    build_gru_2,
    objective='val_accuracy',
    max_trials=30,
    overwrite=True,
    directory='my_dir2',
    project_name='gru')

In [16]:
tuner2.search(X_train, y_train, validation_data=(X_test, y_test), epochs=50, callbacks=[tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10)])

Trial 30 Complete [00h 03m 38s]
val_accuracy: 0.7589403986930847

Best val_accuracy So Far: 0.7741721868515015
Total elapsed time: 00h 44m 42s


In [17]:
tuner2.get_best_hyperparameters()[0].values

{'gru_units': 128,
 'dense_dropout': 0.024211111743395557,
 'learning_rate': 0.004014421368861155,
 'batch_size': 16}

In [33]:
tuner3 = Custom_Bayesian(
    build_gru_3,
    objective='val_accuracy',
    max_trials=30,
    overwrite=True,
    directory='my_dir3',
    project_name='gru')

In [34]:
tuner3.search(X_train, y_train, validation_data=(X_test, y_test), epochs=50, callbacks=[tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10)])

Trial 30 Complete [00h 00m 51s]
val_accuracy: 0.765562891960144

Best val_accuracy So Far: 0.7821192145347595
Total elapsed time: 01h 35m 40s


In [35]:
tuner3.get_best_hyperparameters()[0].values

{'gru_base_units': 32,
 'dropout': 0.42090879788545155,
 'learning_rate': 0.002967008268835806,
 'batch_size': 64}